In [2]:
# Imports we need.
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import os
import collections

from tensor2tensor import problems
from tensor2tensor.layers import common_layers
from tensor2tensor.utils import trainer_lib
from tensor2tensor.utils import t2t_model
from tensor2tensor.utils import registry
from tensor2tensor.utils import metrics

import urllib

# Enable TF Eager execution
tfe = tf.contrib.eager
tfe.enable_eager_execution()

# Other setup
Modes = tf.estimator.ModeKeys

W0627 14:08:25.631115 4626744768 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W0627 14:08:27.883018 4626744768 deprecation_wrapper.py:119] From /Users/samiraabnar/Codes/tensor2tensor/tensor2tensor/utils/expert_utils.py:68: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.

W0627 14:08:31.274685 4626744768 deprecation_wrapper.py:119] From /Users/samiraabnar/Codes/tensor2tensor/tensor2tensor/rl/gym_utils.py:235: The name tf.logging.info is deprecated. Please use tf.compat.v1.logging.info instead.

W0627 14:08:31.311038 4626744768 deprecation_wrapper.py:119] From /Users/samiraabnar/Codes/tensor2tensor/tensor2tensor/u

In [2]:
# Link to data from GLUE: https://gluebenchmark.com/tasks
URLS = [
'http://cogcomp.org/Data/QA/QC/train_5500.label',
'http://cogcomp.org/Data/QA/QC/TREC_10.label'
]

tmp_dir = "tmp"
trec6_train_filename = "train_5500.label"
trec6_test_filename = "TREC_10.label"
trec6_finalpath = os.path.join(tmp_dir)

if not os.path.isdir(trec6_finalpath):
    os.makedirs(trec6_finalpath)

# Download train
urllib.request.urlretrieve(URLS[0], filename=os.path.join(trec6_finalpath,trec6_train_filename))
# Download test
urllib.request.urlretrieve(URLS[1], filename=os.path.join(trec6_finalpath, trec6_test_filename))

('tmp/TREC_10.label', <http.client.HTTPMessage at 0x13217a4e0>)

In [4]:
!mkdir ../data/trec6

data_dir='../data/trec6'
tmp_dir='../tmp/trec6'

# Fetch the problem
problem = problems.problem("trec6")
# The generate_data method of a problem will download data and process it into
# a standard format ready for training and evaluation.
problem.generate_data(data_dir, tmp_dir)

mkdir: ../data/trec6: File exists


W0627 14:08:37.323793 4626744768 deprecation_wrapper.py:119] From /Users/samiraabnar/Codes/tensor2tensor/tensor2tensor/data_generators/generator_utils.py:132: The name tf.gfile.Exists is deprecated. Please use tf.io.gfile.exists instead.



In [6]:
# Get the encoders from the problem
encoders = problem.feature_encoders(data_dir)

# Setup helper functions for encoding and decoding
def encode(input_str, output_str=None, mode="inputs"):
  """Input str to features dict, ready for inference"""
  inputs = encoders[mode].encode(input_str) + [1]  # add EOS id
  batch_inputs = tf.reshape(inputs, [1, -1, 1])  # Make it 3D.
  return {"inputs": batch_inputs}

def decode(integers, mode="inputs"):
  """List of ints to str"""
#  integers = list(np.squeeze(integers))
#  if 1 in integers:
#    integers = integers[:integers.index(1)]
  return encoders[mode].decode(np.squeeze(integers))



In [7]:
example = tfe.Iterator(problem.dataset(Modes.EVAL, data_dir)).next()
example_inputs = example["inputs"]
example_label = example["targets"]

print(example)
print("input:", decode(example_inputs, mode="inputs"))
print("Label: %s" %  encoders["targets"].decode(example_label))

{'batch_prediction_key': <tf.Tensor: id=119, shape=(1,), dtype=int64, numpy=array([0])>, 'inputs': <tf.Tensor: id=120, shape=(5,), dtype=int64, numpy=array([   4,    5, 5273,    2,    1])>, 'targets': <tf.Tensor: id=121, shape=(1,), dtype=int64, numpy=array([0])>}
input: What is leukemia ?<EOS>
Label: DESC
